# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

In [4]:
from pyspark.ml.recommendation import ALS

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
data = spark.read.csv('movielens_ratings.csv', header=True, inferSchema=True)

In [7]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [10]:
# Split the data
train_Data, test_Data = data.randomSplit([0.8,0.2])

# initialize the Alternative Least Square Model
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

# train the model
model = als.fit(train_Data)

In [11]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test_Data)
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27|  2.4613729|
|     31|   1.0|     5|   4.248049|
|     31|   2.0|    25|  2.2851958|
|     31|   1.0|    29|  3.2532406|
|     85|   1.0|    28|  6.1530414|
|     85|   1.0|    12|  1.6653675|
|     85|   5.0|     8|  2.7684357|
|     85|   1.0|    25|    0.32968|
|     65|   1.0|    28|-0.05416511|
|     65|   1.0|    19|   2.078913|
|     53|   5.0|     8|  1.3372827|
|     53|   1.0|    23| -0.9566994|
|     53|   1.0|     7|  3.1025424|
|     53|   3.0|    14|  2.5378532|
|     78|   1.0|    27| 0.17724317|
|     78|   1.0|     1|  1.1686282|
|     78|   1.0|     4|   0.664401|
|     78|   1.0|     8|  0.6311631|
|     78|   1.0|    24| 0.37471634|
|     78|   1.0|     2|-0.07108559|
+-------+------+------+-----------+
only showing top 20 rows



In [12]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating',
                                predictionCol='prediction')

In [13]:
rmse = evaluator.evaluate(predictions)

In [19]:
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.69181402911638


The RMSE described our error in terms of the stars rating column.


So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [15]:
single_user = test_Data.filter(test_Data['userId'] == 11).select(['movieId', 'userId'])

In [20]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     11|    11|
|     13|    11|
|     21|    11|
|     35|    11|
|     41|    11|
|     67|    11|
|     81|    11|
|     88|    11|
|     89|    11|
|     94|    11|
+-------+------+



In [17]:
recommendations = model.transform(single_user)

In [18]:
recommendations.orderBy('prediction', ascrending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     89|    11|-1.8498778|
|     88|    11|-1.7619332|
|      9|    11|-1.5225363|
|     81|    11| 0.3021875|
|     67|    11| 0.5816896|
|     41|    11|0.73185813|
|     21|    11| 1.4683943|
|     94|    11| 2.5970733|
|     13|    11| 2.6232445|
|     35|    11| 3.3231866|
|     11|    11| 3.5483043|
+-------+------+----------+

